In [4]:
from langchain_core.runnables import RunnableLambda

# A RunnableSequence constructed using the `|` operator
sequence = RunnableLambda(lambda x: x + 3) | RunnableLambda(lambda x: x * 2)
sequence.invoke(1) # 4
# sequence.batch([1, 2, 3]) # [4, 6, 8]


# # A sequence that contains a RunnableParallel constructed using a dict literal
# sequence = RunnableLambda(lambda x: x + 1) | {
#     'mul_2': RunnableLambda(lambda x: x * 2),
#     'mul_5': RunnableLambda(lambda x: x * 5),
#     'mul_10': RunnableLambda(lambda x: x * 10)
# }
# sequence.invoke(1) # {'mul_2': 4, 'mul_5': 10}

8

In [7]:
from langchain_core.runnables import RunnableLambda

import random

def add_one(x: int) -> int:
    return x + 1


def buggy_double(y: int) -> int:
    '''Buggy code that will fail 70% of the time'''
    if random.random() > 0.3:
        print('This code failed, and will probably be retried!')
        raise ValueError('Triggered buggy code')
    return y * 2

sequence = (
    RunnableLambda(add_one) |
    RunnableLambda(buggy_double).with_retry( # Retry on failure
        stop_after_attempt=10,
        wait_exponential_jitter=False
    )
)
sequence.invoke(1) # 4
print(sequence.input_schema.schema()) # Show inferred input schema
print(sequence.output_schema.schema()) # Show inferred output schema

This code failed, and will probably be retried!
This code failed, and will probably be retried!
This code failed, and will probably be retried!
{'title': 'add_one_input', 'type': 'integer'}
{'title': 'buggy_double_output', 'type': 'integer'}
